In [ ]:
import torch
import torch.nn as nn

from torch.optim import Adam, SGD
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

import random
from PIL import Image
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt


import pylab as pl
from IPython import display

## Segmentació d'objectes emprant Unet

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Una manera diferent d'activar la GPU

**Feina a fer**

Un cop teniu els conjunts de dades creats heu de comprovar que les imatges que es corresponen amb les etiquetes tenen la informació correcta, feis una visualització.

In [ ]:
import os
# Dades entrenament
path_train = # PUT YOUR PATH
files = os.listdir(path_train)
img_files = list([f"{path_train}{p}" for p in files if p.endswith('.png')])[:500]
label_files = list([f"{path_train}gt/{p}" for p in files if p.endswith('.png')])[:500]
print("total training images", len(img_files))

# Dades validacio

path_val = # PUT YOUR PATH
files = os.listdir(path_val)
img_files_val = list([f"{path_val}{p}" for p in files if p.endswith('.png')])
label_files_val = list([f"{path_val}gt/{p}" for p in files if p.endswith('.png')])
print("total test images", len(img_files_val))

In [ ]:
train_batch_size = 64
test_batch_size = 100

# Definim una seqüència (composició) de transformacions 
transform=transforms.Compose([
    transforms.ToTensor()
    ## TODO: Put if necessary
    ])

# Constructor del dataset.
class Formes(Dataset):
    def __init__(self, images, labels, transform):
        super().__init__()
        self.paths = images
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform

    def __len__(self): 
        return self.len
    
                
    def __getitem__(self, index):

        path = self.paths[index]
        label = self.labels[index]
        image = cv2.imread(path) #, cv2.IMREAD_GRAYSCALE)  # Depén de vosaltres
        
        image = self.transform(image)
        label_img = cv2.imread(label)

        merged_image = cv2.add(label_img[:,:,0], cv2.add(label_img[:,:,1], label_img[:,:,2]))
        
        label_img = self.transform(merged_image)
        return image,  #### TODO
    
train_data = Formes(img_files, label_files, transform)
val_data = Formes(img_files_val, label_files_val, transform)

train_loader = torch.utils.data.DataLoader(train_data, train_batch_size)
val_loader = torch.utils.data.DataLoader(val_data, test_batch_size)

In [ ]:
iterador =  iter(val_loader) 

In [ ]:
features, labels = next(iterador)

print("Saber l'estructura del batch us ajudarà: ")
print(f"Feature batch shape: {features.size()}")
print(f"Labels batch shape: {labels.size()}")


In [ ]:
# TODO: mostrar una imatge del batch i devora mostrar l'imatge que fa d'etiqueta



## Definició de la xarxa

Podem observar com es pot emprar l'orientació a objectes de **Python** per crear una xarxa de manera ordenada, és interessant analitzar aquest codi amb detall ja que en podem aprendre molt:

Aquí tenim 2 capes noves que ens ajudaràn a construïr la nova arquitectura:

- [ConvTranspose2d](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html): Aplica un operador de convolució transposat 2D sobre una imatge d'entrada composta per diversos plans d'entrada.    [Exemples gràfics](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md).
- [BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html): És una tècnica utilitzada en l'entrenament de xarxes neuronals artificials per a estabilizar i accelerar el procés de convergència durant l'entrenament. Bàsicament, durant l'entrenament d'una red neuronal, els valors d'entrada de cada capa poden canviar a mesura que els paràmetres de les capes anteriors s'actualitzen. Això pot fer que l'entrenament sigui més lento o inestable. La normalització per lots resuelve això normalitzant les activacions de cada capa abans de passar a la següent capa.

In [ ]:
from collections import OrderedDict

class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        
        ## CODER
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")
        
        ## DECODER
        
        
        # TODO: Construeix el teu decoder
        
        self.upconv4 = nn.ConvTranspose2d(¿?, ¿?, kernel_size=2, stride=2)  # Empra aquesta capa com exemple
        
        

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)

        
        return torch.sigmoid()  # Aplicarem una sigmoide a la sortida de la xarxa -> TODO: Recordar el que ha dit en biel a classe

    
    # Ara ja podem començar a fer coses amb cara i ulls
    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

## Entrenament

Per fer tasques de segmentació, una de les funcions de pèrdua que podem emprar és el _Diceloss_ (intersecció vs unió):  El coeficient de _Dice_ s'utilitza habitualment en tasques de segmentació d'imatges com a mesura de la superposició entre les màscares de segmentació entre la predicció i el _ground truth_. El  _Diceloss_, és el complementari del coeficient de _Dice_, es pot utilitzar com a funció de pèrdua per entrenar models per a tasques de segmentació.

Dice Coefficient=$ = 2 \times \frac{|X \cap Y|}{|X| + |Y|}$



On:

- $X$ és la màscara de segmentació prevista.
- $Y$ és la màscara de segmentació de la veritat del sòl.
- $∣⋅∣$ denota la cardinalitat o el nombre d'elements d'un conjunt.

In [ ]:
class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 0.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
            y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

El bucle d'entrenament és diferent al que estau acostumats a veure en l'assignatura, s'assembla molt més als propis tutorials de _Pytorch_.

A més s'aprofita per introduir la visualització de resultats de forma dinàmica usant la llibreria [tqdm](https://github.com/tqdm/tqdm) i la llibreria _matplotlib_

In [ ]:
use_cuda = True
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [ ]:
epochs = 40

model = UNet().to(device)

optim = Adam(model.parameters(), lr=1e-3)
criterion = DiceLoss() 

t_loss = np.zeros((epochs))
v_loss = np.zeros((epochs))

pbar = tqdm(range(1, epochs+1)) # tdqm permet tenir text dinàmic

for epoch in pbar:
    
    train_loss = 0 
    val_loss = 0  
    
    model.train()                                                  
    for batch_num, (input_img, target) in enumerate(train_loader, 1):   
        input_img= input_img.to(device)
        target = target.to(device)
        
        output = model(input_img)
        loss = criterion(output, target)
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()     
        
        train_loss += loss.item()    
                                                       
    model.eval()   
    with torch.no_grad():                                          
        for input_img, target in val_loader: 
            input_img = input_img.to(device)
            target = target.to(device)
            
            output = model(input_img)                                   
            loss = criterion(output, target)   
            val_loss += loss.item()  
    
    # RESULTATS
    train_loss /= len(train_loader)
    t_loss[epoch-1] = train_loss
    
    val_loss /= len(val_loader)   
    v_loss[epoch-1] = val_loss
    
    # VISUALITZACIO DINAMICA
    plt.figure(figsize=(12, 4))
    pl.plot(t_loss[:epoch], label="train")
    pl.plot(v_loss[:epoch], label="validation")
    pl.legend()
    pl.xlim(0, epochs)
    pl.xticks(range(0,epochs,1),range(1,epochs+1,1))
    
    display.clear_output(wait=True)
    display.display(pl.gcf())
    plt.close()

    pbar.set_description(f"Epoch:{epoch} Training Loss:{train_loss} Validation Loss:{val_loss}")

Guardam el model, d'aquesta manera no es necessari fer l'entrenament a classe:

In [ ]:
torch.save(model.state_dict(), "unet.pt")

## Avaluació

Carregam el model

In [ ]:
mmodel =  UNet().to(device)
mmodel.load_state_dict(torch.load("unet.pt"))
mmodel.eval();

**Feina a fer**

Visualitzar exemples de segmentació.

[agraïments Unet](https://github.com/mateuszbuda/brain-segmentation-pytorch)